In [15]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

# Part 2 Data Retrieval
Use the `requests` library and the USGS's API (https://earthquake.usgs.gov/fdsnws/event/1/) to retrieve information about all recorded earthquakes that occurred in Tennessee since 1900.
Answer the following questions:

In [12]:
url = 'https://earthquake.usgs.gov/fdsnws/event/1/catalogs'

In [13]:
r = requests.get(url)

In [16]:
r.text

'<?xml version="1.0"?>\n<Catalogs><Catalog>38457511</Catalog><Catalog>=c</Catalog><Catalog>ak</Catalog><Catalog>at</Catalog><Catalog>atlas</Catalog><Catalog>av</Catalog><Catalog>cdmg</Catalog><Catalog>cgs</Catalog><Catalog>choy</Catalog><Catalog>ci</Catalog><Catalog>cidev</Catalog><Catalog>dr</Catalog><Catalog>duputel</Catalog><Catalog>eqh</Catalog><Catalog>ew</Catalog><Catalog>ew_dm</Catalog><Catalog>gcmt</Catalog><Catalog>gsc</Catalog><Catalog>hv</Catalog><Catalog>id</Catalog><Catalog>is</Catalog><Catalog>iscgem</Catalog><Catalog>iscgemsup</Catalog><Catalog>ismpkansas</Catalog><Catalog>ld</Catalog><Catalog>mb</Catalog><Catalog>nc</Catalog><Catalog>ne</Catalog><Catalog>nm</Catalog><Catalog>nn</Catalog><Catalog>official</Catalog><Catalog>official19631013051759_30</Catalog><Catalog>ok</Catalog><Catalog>pr</Catalog><Catalog>pt</Catalog><Catalog>sc</Catalog><Catalog>se</Catalog><Catalog>unknown</Catalog><Catalog>us</Catalog><Catalog>ushis</Catalog><Catalog>uu</Catalog><Catalog>uw</Catalog

In [17]:
catalogs = bs(r.text)

In [18]:
catalogs

<?xml version="1.0"?><html><body><catalogs><catalog>38457511</catalog><catalog>=c</catalog><catalog>ak</catalog><catalog>at</catalog><catalog>atlas</catalog><catalog>av</catalog><catalog>cdmg</catalog><catalog>cgs</catalog><catalog>choy</catalog><catalog>ci</catalog><catalog>cidev</catalog><catalog>dr</catalog><catalog>duputel</catalog><catalog>eqh</catalog><catalog>ew</catalog><catalog>ew_dm</catalog><catalog>gcmt</catalog><catalog>gsc</catalog><catalog>hv</catalog><catalog>id</catalog><catalog>is</catalog><catalog>iscgem</catalog><catalog>iscgemsup</catalog><catalog>ismpkansas</catalog><catalog>ld</catalog><catalog>mb</catalog><catalog>nc</catalog><catalog>ne</catalog><catalog>nm</catalog><catalog>nn</catalog><catalog>official</catalog><catalog>official19631013051759_30</catalog><catalog>ok</catalog><catalog>pr</catalog><catalog>pt</catalog><catalog>sc</catalog><catalog>se</catalog><catalog>unknown</catalog><catalog>us</catalog><catalog>ushis</catalog><catalog>uu</catalog><catalog>uw

In [20]:
cat_list = catalogs.find_all('catalog')

In [21]:
cat_list

[<catalog>38457511</catalog>,
 <catalog>=c</catalog>,
 <catalog>ak</catalog>,
 <catalog>at</catalog>,
 <catalog>atlas</catalog>,
 <catalog>av</catalog>,
 <catalog>cdmg</catalog>,
 <catalog>cgs</catalog>,
 <catalog>choy</catalog>,
 <catalog>ci</catalog>,
 <catalog>cidev</catalog>,
 <catalog>dr</catalog>,
 <catalog>duputel</catalog>,
 <catalog>eqh</catalog>,
 <catalog>ew</catalog>,
 <catalog>ew_dm</catalog>,
 <catalog>gcmt</catalog>,
 <catalog>gsc</catalog>,
 <catalog>hv</catalog>,
 <catalog>id</catalog>,
 <catalog>is</catalog>,
 <catalog>iscgem</catalog>,
 <catalog>iscgemsup</catalog>,
 <catalog>ismpkansas</catalog>,
 <catalog>ld</catalog>,
 <catalog>mb</catalog>,
 <catalog>nc</catalog>,
 <catalog>ne</catalog>,
 <catalog>nm</catalog>,
 <catalog>nn</catalog>,
 <catalog>official</catalog>,
 <catalog>official19631013051759_30</catalog>,
 <catalog>ok</catalog>,
 <catalog>pr</catalog>,
 <catalog>pt</catalog>,
 <catalog>sc</catalog>,
 <catalog>se</catalog>,
 <catalog>unknown</catalog>,
 <cata

In [24]:
cats = [cat.text.strip() for cat in cat_list]

In [25]:
cats

['38457511',
 '=c',
 'ak',
 'at',
 'atlas',
 'av',
 'cdmg',
 'cgs',
 'choy',
 'ci',
 'cidev',
 'dr',
 'duputel',
 'eqh',
 'ew',
 'ew_dm',
 'gcmt',
 'gsc',
 'hv',
 'id',
 'is',
 'iscgem',
 'iscgemsup',
 'ismpkansas',
 'ld',
 'mb',
 'nc',
 'ne',
 'nm',
 'nn',
 'official',
 'official19631013051759_30',
 'ok',
 'pr',
 'pt',
 'sc',
 'se',
 'unknown',
 'us',
 'ushis',
 'uu',
 'uw']